# 순서

### 1. module import 및 driver option 설정

### 2. 뉴스 url 크롤링

### 3. 뉴스 title, date, content 스크래핑

In [28]:
from tqdm import notebook
import pandas as pd
from IPython.display import clear_output
import requests
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
import os

options = webdriver.ChromeOptions()
options.add_argument('headless')  # 크롬 브라우저 팝업 뜨는 거 안보이게 해줌 ㅎㅎ
options.add_argument('window-size=1920x1080') # 하라는데 왜 했는 지 모르겠습니다.
options.add_argument("disable-gpu") # 브라우저 띄울 때 gpu 쓰는 거 막아줘서 속도 향상



press = {
    '매일경제' : 1009,
    '머니투데이' : 1008,
    '서울경제' : 1011,
    '아시아경제' : 1277,
    '아주경제' : 2090,
    '파이낸셜뉴스' : 1014,
    '한국경제' : 1015, 
    '헤럴드경제' : 1016
}             # 다른 언론사 코드 궁금하면 네이버 뉴스에서 언론사 하나씩 선택해보면서 url주소 checked 뒤에 있는 숫자 확인

search_keyword = input('궁금한 종목을 입력 해주세요!: ') #종목 입력 (다른거 기사 궁금하면 다른거 입력해서 해도 됨)

if not os.path.isdir('urls'):                                                           
    os.mkdir('urls')
if not os.path.isdir('contents'): 
    os.mkdir('contents')
# 경로 잡아주기


# url 주소를 저장할 리스트 생성
print('urls: ')
for i in notebook.tqdm(press):
    url_list = []
    
    for page in range(1, 11, 10):
        site = 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query='+search_keyword+'&sort=1&photo=0&field=0&pd=3&ds=2020.06.01&de=2021.06.30&mynews=1&office_type=1&office_section_code=3&news_office_checked='+str(press.get(i))+'&nso=so:dd,p:from20200601to20210630,a:all&start='+str(page)
        driver = webdriver.Chrome('./chromedriver.exe', chrome_options=options)
        driver.get(site)
        # 뉴스 url 수집하기
        things = driver.find_elements_by_link_text('네이버뉴스')  
        
        for thing in things:
            url = thing.get_attribute('href')
            url_list.append(url)

        driver.close()
    df = pd.DataFrame({"url":url_list})
    df.to_csv('urls/'+search_keyword+'_'+i+'_urls.csv')
    print(i+" 저장완료")
    
print('')
print('url 스크래핑 완료')
print('')


print('contents: ')

for t in notebook.tqdm(press):
    
    df = pd.read_csv('urls/'+search_keyword+'_'+t+'_urls.csv')
    urls=df.url[:]
    data_dict = {
    'date' : [],
    'title' : [],
    'contents' : []
}
    
    for i in range(len(urls)):
        driver = webdriver.Chrome("./chromedriver.exe", chrome_options=options)
        driver.get(df['url'][i])

        # 기사 시간 스크랩
        try:
            driver.find_element_by_css_selector('.t11').click()
            data_dict['date'].append(driver.find_element_by_css_selector('.t11').text.strip())
        except NoSuchElementException:
            pass
        
        # 기사 제목 스크랩
        try:
            driver.find_element_by_css_selector('.tts_head').click()
            data_dict['title'].append(driver.find_element_by_css_selector('.tts_head').text.strip())
        except NoSuchElementException:
            pass
        
        # 기사 내용 스크랩
        try:
            driver.find_element_by_css_selector('.tts_head').click()
            data_dict['contents'].append(driver.find_element_by_css_selector('._article_body_contents').text.strip())
        except NoSuchElementException:
            pass

        driver.close()

    df1=pd.DataFrame(data_dict)
    df1.to_csv('contents/'+search_keyword+'_'+t+'_news_contents.csv', encoding='utf-8-sig', index=False)
    print(t+" 저장완료")

print('')
print('본문 스크래핑 완료')

궁금한 종목을 입력 해주세요!: 애플
urls: 


  0%|          | 0/8 [00:00<?, ?it/s]

<ipython-input-28-5c39df692bf6>:43: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome('./chromedriver.exe', chrome_options=options)


매일경제 저장완료
머니투데이 저장완료
서울경제 저장완료
아시아경제 저장완료
아주경제 저장완료
파이낸셜뉴스 저장완료
한국경제 저장완료
헤럴드경제 저장완료

url 스크래핑 완료

contents: 


  0%|          | 0/8 [00:10<?, ?it/s]

<ipython-input-28-5c39df692bf6>:75: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome("./chromedriver.exe", chrome_options=options)


매일경제 저장완료
머니투데이 저장완료
서울경제 저장완료
아시아경제 저장완료
아주경제 저장완료
파이낸셜뉴스 저장완료
한국경제 저장완료
헤럴드경제 저장완료

본문 스크래핑 완료


In [27]:
df=pd.read_csv('contents/삼성SDI_매일경제_news_contents.csv',encoding='utf-8-sig')
df

,date,title,contents
0,2021.06.29. 오후 5:40,이달에도 매도일색 국민연금 이 종목은 담았다,6월 보유종목 비중 분석\n\n하반기 車수요 증가 전망에\n현대위아·SNT모티브 등...
1,2021.06.28. 오후 5:45,배터리·백신 설비에 1조 투자땐 1000억 세금감면,신성장 기술에 稅혜택\n\n생산시설 투자 공제율 확대\n대기업 R&D 공제도 40%...
2,2021.06.27. 오후 5:40,"""올해에만 24조 사들였는데 수익률 -2.2%"" 개미 삼성전자에 울었다…카카오는 9...",상반기 개인 순매수 톱10 분석\n\n삼성전자 순매수 24조원 1위\n수익률 작년 ...
3,2021.06.26. 오전 11:02,"""라이벌 대만 의식?""...삼성전자 SK하이닉스 직원 '모더나' 맞는다 [위클리반도체]",인천국제공항을 통해 이달 하순 국내에 들어온 모더나 코로나 19 백신. [연합뉴스]...
4,2021.06.25. 오후 5:35,"삼성 임직원, 내달 27일 모더나 백신 자체 접종…협력사 직원 포함","만18~59세·협력사직원 포함\n현대차·SK이노베이션 조율중\n\n삼성전자, 현대자..."
5,2021.06.25. 오후 2:11,"시민단체 7곳 ""공정위, 삼성웰스토리 부당지원 추가 고발해야""","""책임자·계열사에 '솜방망이' 처벌""\n일감몰아주기는 공정위 전속고발제 대상\n""중..."
6,2021.06.24. 오후 9:12,카카오 네이버가 다했네…코스피 이달 시총 증가 40% 차지,"23일 카카오와 NAVER는 각각 16만9500원, 42만3500원으로 신고가를 기..."
7,2021.06.24. 오후 5:47,"공정위, 삼성에 2349억원 과징금","급식 몰아준 삼성전자 檢고발\n계열사 급식 100% 수의계약\n삼성 ""복지를 부당지..."
8,2021.06.24. 오후 5:43,"삼성 ""매출 1조 웰스토리가 그룹 자금줄이라니"" 공정위 제재 반박","공정위 역대 최대 과징금에 삼성 ""행정소송 나설것""\n\n""年 1천억 영업익 웰스토..."
9,2021.06.24. 오후 1:55,"""직원 식사 잘 준 것 뿐""…삼성, 공정위 2천억 과징금 행정소송 예고",[사진 = 이충우 기자]\n삼성웰스토리에 계열사 급식 물량을 물어줘 공정거래위원회로...
